In [ ]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from datetime import datetime # datetime library to handle dates in datasets

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.plotly as py
from plotly.graph_objs import *

LIMIT = 30

In [ ]:
def search_address_df(address, SEARCH_QUERY):
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    LATITUDE = location.latitude
    LONGITUDE = location.longitude
    url="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, LATITUDE, LONGITUDE, VERSION, SEARCH_QUERY, LIMIT)
    results = requests.get(url).json()
    venues = results["response"]["venues"]
    dataframe = json_normalize(venues)
    filtered_columns = ['name', 'url'] + [col for col in dataframe.columns if col.startswith('location.')]
    dataframe_filtered = dataframe.ix[:, filtered_columns]
    dataframe_filtered.columns = [column.split(".")[-1] for column in dataframe_filtered.columns]
    return dataframe_filtered.head()

search_address_df("525 Stockton St, San Francisco CA", "tea")

In [ ]:
def search_address_plot(address, SEARCH_QUERY):
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    LATITUDE = location.latitude
    LONGITUDE = location.longitude
    url="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, LATITUDE, LONGITUDE, VERSION, SEARCH_QUERY, LIMIT)
    results = requests.get(url).json()
    venues = results["response"]["venues"]
    dataframe = json_normalize(venues)
    filtered_columns = ['name', 'url'] + [col for col in dataframe.columns if col.startswith('location.')]
    dataframe_filtered = dataframe.ix[:, filtered_columns]
    dataframe_filtered.columns = [column.split(".")[-1] for column in dataframe_filtered.columns]    
    data = Data(
        [
            Scattermapbox(
                lat=[LATITUDE] + list(dataframe_filtered.lat),
                lon=[LONGITUDE] + list(dataframe_filtered.lng),
                mode='markers',
                marker=Marker(
                    size=10,
                    color=["red"] + ["blue"]*len(dataframe_filtered)
                ),
                text=["Home"] + list(dataframe_filtered.name),
            )
        ]
    )

    # define the layout and centre the map around Galvanize
    layout = Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=int(LATITUDE),
                lon=int(LONGITUDE)
            ),
            pitch=0,
            zoom=5
        ),
    )

    # plot the data
    fig = dict(data=data, layout=layout)
    return py.iplot(fig, filename='Galvanize Search Mapbox')
search_address_plot("44 Tehama St San Francisco, CA", "chinese")

In [ ]:
from io import StringIO
import requests
import json
import pandas as pd



In [ ]:
df_data_2 = pd.read_csv(get_object_storage_file_with_credentials_09bd790c49f84532a1fc326f9db6c10c('DefaultProjectyuneleougmailcom', 'ExerciseLog.csv'))
df_data_2 = df_data_2.rename(columns={'occurredDate':'Date'})
df_data_2 = df_data_2.set_index('Date')

In [ ]:
df_data_3 = pd.read_csv(get_object_storage_file_with_credentials_09bd790c49f84532a1fc326f9db6c10c('DefaultProjectyuneleougmailcom', 'ConvLog_flagged.csv'))
df_data_3 = df_data_3.rename(columns={'occurredDate':'Date'})
df_data_3 = df_data_3.set_index('Date')

In [ ]:
import json
from watson_developer_cloud import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(username=,
   password=,
   version=)

In [ ]:
def analyze_tones(df, row):
    df3 = pd.DataFrame(index = ['Date'], columns = ['notes', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Analytical',
       'Confident', 'Tentative', 'Openness', 'Conscientiousness',
       'Extraversion', 'Agreeableness', 'Emotional Range'])
    output = tone_analyzer.tone(text=df['notes'][row])
    emotional_tone = pd.DataFrame(output['document_tone']['tone_categories'][0]['tones']).set_index('tone_name').drop('tone_id', 1)
    language_tone = pd.DataFrame(output['document_tone']['tone_categories'][1]['tones']).set_index('tone_name').drop('tone_id', 1)
    social_tone = pd.DataFrame(output['document_tone']['tone_categories'][2]['tones']).set_index('tone_name').drop('tone_id', 1)
    new = pd.concat([emotional_tone, language_tone, social_tone])
    new2 = new.T
    new2['Date'] = df['notes'].index[row]
    new2 = new2.set_index('Date')
    new2['notes'] = df['notes'][row]
    return new2


In [ ]:
conversation_analysis_2 = pd.DataFrame()
for i in range (159, 714):
    if isinstance(df_data_3.iloc[i]['notes'], str) == True:
        conversation_analysis_2 = pd.concat([conversation_analysis_2, analyze_tones(df_data_3, i)])
    else:
        continue
    print(i)

In [ ]:
conversation_analysis = pd.concat([conversation_analysis, conversation_analysis_2])

In [ ]:
exercise_analysis = pd.DataFrame()
for i in range(len(df_data_2)):
    exercise_analysis = pd.concat([exercise_analysis, analyze_tones(df_data_2, i)])
    print(i)

In [48]:
conversation_analysis

tone_name,Anger,Disgust,Fear,Joy,Sadness,Analytical,Confident,Tentative,Openness,Conscientiousness,Extraversion,Agreeableness,Emotional Range,notes
Date,,,,,,,,,,,,,,
7/9/12 13:41,0.188075,0.046503,0.110975,0.585304,0.499757,0.624968,0.000000,0.683401,0.387282,0.994770,0.735788,0.861806,0.815301,"Hello ,\r\rWelcome to Retrofit! I look forward..."
7/11/12 18:51,0.016908,0.005340,0.125800,0.532970,0.148875,0.668742,0.000000,0.000000,0.790319,0.875935,0.653570,0.864779,0.958125,Hi \r\rIt was great talking with you today. He...
7/12/12 0:19,0.169754,0.011024,0.013126,0.317294,0.316928,0.000000,0.000000,0.000000,0.283972,0.514022,0.475393,0.625826,0.470029,Got my devices tonight. Will be setting them up.
7/12/12 13:23,0.088146,0.036602,0.042221,0.474091,0.188053,0.000000,0.000000,0.000000,0.618793,0.263554,0.696881,0.571078,0.135875,Weighed myself with new scale this morning. M...
7/12/12 13:23,0.140132,0.056817,0.205580,0.017940,0.626761,0.774940,0.000000,0.000000,0.264049,0.449911,0.188743,0.155863,0.882799,", Is it on a hard surface? How do you know it ..."
7/12/12 13:50,0.145483,0.086413,0.161832,0.037185,0.597967,0.336637,0.693873,0.000000,0.731579,0.004897,0.553265,0.128166,0.297428,"Sorry, I meant my personal scale was off by 8 ..."
7/13/12 14:28,0.090198,0.016371,0.074719,0.554518,0.106778,0.000000,0.000000,0.000000,0.029890,0.377126,0.496892,0.634139,0.648452,Whoops! Make that Thursday 11 pm
7/14/12 1:38,0.084531,0.029158,0.160677,0.338593,0.229032,0.000000,0.000000,0.990986,0.493675,0.847578,0.371267,0.483703,0.897750,Just got back from the grocery store. Going t...
7/14/12 15:53,0.088123,0.071372,0.646238,0.003877,0.268485,0.657149,0.755896,0.000000,0.294422,0.007134,0.000041,0.169899,0.004695,changed my fitbit to be more sensitive yesterd...


In [49]:
exercise_analysis

tone_name,Anger,Disgust,Fear,Joy,Sadness,Analytical,Confident,Tentative,Openness,Conscientiousness,Extraversion,Agreeableness,Emotional Range,notes
Date,,,,,,,,,,,,,,
7/12/12 0:19,0.124358,0.086222,0.112084,0.395038,0.137991,0.000000,0.000000,0.000000,0.266421,0.274438,0.543198,0.599492,0.294410,15 minutes gardening
7/13/12 0:34,0.222013,0.036744,0.409953,0.065821,0.280929,0.724236,0.000000,0.000000,0.060545,0.260868,0.476688,0.569809,0.344730,I burned 74 calories at my bass guitar lesson :-)
7/13/12 23:13,0.310003,0.124179,0.196353,0.066869,0.333633,0.000000,0.000000,0.000000,0.284713,0.274153,0.543427,0.598730,0.303717,"Rowing from 5 - 5:30, 5000 meters"
7/15/12 20:01,0.476461,0.123835,0.161781,0.068348,0.203183,0.000000,0.000000,0.000000,0.204990,0.274443,0.543189,0.599511,0.276883,30 mins rowing
7/20/12 0:27,0.045848,0.039163,0.218841,0.504820,0.043852,0.694442,0.000000,0.000000,0.012813,0.589637,0.435404,0.353043,0.248982,Had a private Pilates lesson this afternoon. ...
7/22/12 4:56,0.087603,0.065815,0.336669,0.149884,0.296045,0.000000,0.000000,0.909883,0.451440,0.416862,0.551314,0.431914,0.538166,Lots of work around the house and in the yard ...
7/23/12 1:37,0.426520,0.035311,0.253778,0.007620,0.361225,0.405753,0.000000,0.902459,0.009909,0.349288,0.232094,0.386837,0.299104,Had another Pilates class this afternoon. Wen...
7/25/12 19:54,0.154173,0.107287,0.272021,0.118809,0.321226,0.000000,0.000000,0.855910,0.118659,0.384609,0.475464,0.191114,0.185980,Went to an 11 am Pilates class today. Really ...
7/28/12 23:19,0.107765,0.143097,0.069684,0.487203,0.061643,0.000000,0.000000,0.000000,0.039073,0.383264,0.422020,0.587213,0.329141,Swam for 45 minutes this afternoon


In [ ]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

#df = pd.DataFrame(data = [[1,2],[3,4]], columns=['Col 1', 'Col 2'])
create_download_link(exercise_analysis)

In [ ]:
create_download_link(conversation_analysis)